<img width="50px" align="left" style="margin-right:20px" src="/extra/images/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 2

# Построить content-based рекомендательную систему товаров интернет-магазина ozon.ru

<img width="200px" align="left" src="/extra/images/ozon.png">

<a href="https://github.com/newprolab/content_bigdata9"><img align="left" src="/extra/images/npl7.svg"></a>

### Задача

Нужно построить рекомендательный алгоритм наилучшей предсказательной точности по метрике NDCG@100.

<!-- Подробнее о метрике: https://www.kaggle.com/wiki/NormalizedDiscountedCumulativeGain -->

Можно прочесть качественный пост на FastML: http://fastml.com/evaluating-recommender-systems/

Или на русском https://habr.com/company/econtenta/blog/303458/

Как и в первом проекте, мы рекомендуем сделать всё решение в этом ноутбуке. Но, в отличие от первого проекта, где всё-таки были очерчены примерные способы решения, в данном случае вам предоставляется полная свобода для трактования, исследования и выбора подходов к задаче. Более того, нет гарантий, что задача вообще решаема с необходимой точностью.

⏰ **Дедлайн: Воскресенье, 23 июня 2019, 23:59 по Москве**

### Обработка данных на вход

Для выполнения работы вам следует взять данные из HDFS:

```bash
$ hadoop fs -ls /labs/project02 
Found 6 items
-rw-r--r--   3 hdfs hdfs   13550896 2017-04-22 15:04 /labs/project02/catalog_path
-rw-r--r--   3 hdfs hdfs  453269519 2017-04-22 15:04 /labs/project02/catalogs
-rw-r--r--   3 hdfs hdfs 8726734647 2017-04-22 15:05 /labs/project02/item_details_full
-rw-r--r--   3 hdfs hdfs    2240879 2017-04-22 15:05 /labs/project02/ozon_test.txt
-rw-r--r--   3 hdfs hdfs   19097164 2017-04-22 15:05 /labs/project02/ozon_train.txt
-rw-r--r--   3 hdfs hdfs   12906349 2017-04-22 15:05 /labs/project02/ratings
```

Итак, давайте разберёмся, какие входные данные мы имеем:

#### `ozon_train.txt`

Обучающая выборка строчки в json, где для товара `item` мы предоставляем наиболее популярные рекомендации в `true_recoms` (здесь словарь из id рекомендуемого товара и веса — чем больше, тем лучше). Веса означают клики. Текущая рекомендательная система Ozon.ru — это смесь content-based и коллаборативной фильтрации.

Пример:

```
{"item":"24798277","true_recoms":{"24798314":1,"24798279":2,"24798276":4,"24798277":1,"24798280":2}}
```

❗️ В файле есть строчка с 40 000 рекомендаций — это мусор.

#### `ozon_test.txt`

Тестовая выборка.

Пример:

```
{"item": "28759795","recoms": null}
```

#### `item_details_full`

Атрибуты товаров.

Пример:

```
{"id":"4381194","name":"Графиня де Монсоро - В двух томах - Номерованный экземпляр № 84 (подарочное издание)","annotation":"Настоящее издание отпечатано в количестве тысячи пятисот экземпляров, сто из которых изготовлены в переплетах из черной кожи с золотыми обрезами и пронумерованы.  Номер настоящего экземпляра 84.<br>\r\n\"Графиня де Монсоро\" (1846) - одно из самых значительных произведений Александра Дюма. В этом увлекательном авантюрно-историческом романе писатель с замечательным мастерством воскрешает события второй половины XVI века - эпохи религиозных войн и правления Генриха III, последнего короля династии Валуа. История трагической любви благородного графа де Бюсси и прекрасной Дианы де Монсоро развертывается на фоне придворных интриг, политических заговоров и религиозных раздоров. <br>\r\nВ настоящем издании впервые публикуются все 245 иллюстраций выдающегося французского художника Мориса Лелуара, выполненные им для парижского издания 1903 года. Книга дополнена очерком А. И. Куприна \"Дюма-отец\" и обстоятельными комментариями.","parent_id":"18255189"}
```

❗️Под `parent_id` объединяются модификации одного товара (например, разные айфоны).


#### `catalogs`

В каких каталогах лежит товар (может быть несколько записей). 

Пример:

```
{"itemid":"29040016","catalogid":"1179259"}
```

#### `catalog_path`

Пути для каталогов нижнего уровня (в которых лежат товары) в дереве каталогов. Для каждого каталога отдаётся полный путь до корня. 

Пример:

```
{"catalogid":1125630,"catalogpath":[{"1125630":"Изысканные напитки. Сигары"},{"1125623":"Книга - лучший подарок!"},{"1112250":"Архив раздела (Нехудож.лит-ра)"},{"1095865":"Нехудожественная литература"}]}
```

#### `ratings`

Средний рейтинг `itemid` (звёздочки). 

Пример:

```
{“itemid”: 2658646, “rating”:4.0}
```

### Обработка данных на выход

Выходной файл должен иметь следующий формат (пример для одной строчки). Вес товара тем выше, чем выше его близость:

```
{"item": "28759795", "recoms": {"28759801": 1, "28759817": 2, "28759803": 13}}
```

Вы можете использовать любые алгоритмы и их смеси для предсказания рейтингов. Мы будем оценивать точность работы вашего алгоритма, рассчитывая средний NDCG@100 по всем товарам. Это означает, что для каждого `item` в тестовой выборке вы рекомендуете 100 товаров.

Чекер выглядит следующим образом:

```python
lines_number_ok = False
score = 0.0

file_exists = False
test_passed = False
true_recs = {}
lines_number = 0
lines_number_hidden = 0


try:
    if not cli.startcheck():
        exit(-1)

    file_exists = cli.checkfileexists(filename)

    if file_exists:
        with open(cli.getans('test_file_path')) as f:
            for line in f:
                data = json.loads(line)
                true_recs[data['item']] = data['true_recoms']
        lines_number_hidden = len(true_recs.keys())
        
        ndcg_sum = 0
        with open(cli.getfilepath(filename)) as f:
            for line in f:
                lines_number += 1
        
        if lines_number == len(true_recs.keys()):
            lines_number_ok = True
            with open(cli.getfilepath(filename)) as f:
                for line in f:                     
                    data = json.loads(line)     
                    trs = true_recs[data['item']]
                    tmrs = data['recoms']    

                    sorted_trs = sorted(trs.items(), key = lambda x: float(x[1]), reverse = True)
                    sorted_tmrs = sorted(tmrs.items(), key = lambda x: float(x[1]), reverse = True)
                    dcg = 0
                    idcg = 0

                    for i in range(len(trs)):
                        delta = sorted_trs[i][1]
                        if i + 1 != 1:
                            delta = delta / float(math.log(i + 1, 2))
                        idcg += delta

                    for i in range(len(tmrs)):
                        if sorted_tmrs[i][0] in trs:                    
                            delta = trs[sorted_tmrs[i][0]]
                            if i + 1 != 1:
                                delta = delta / float(math.log(i + 1, 2))
                            dcg += delta

                    ndcg_sum += dcg / idcg
            score = ndcg_sum / lines_number

            if score > 0.1:
                test_passed = True

```

Ваши результаты будут заноситься на обновляемую доску лидеров на [странице Проекта 2](http://lk.newprolab.com/lab/project02).

### Проверка

Для автоматической проверки необходимо сохранить заполненный предсказанными вами рейтингами файл `ozon_test.txt` в вашей домашней директории под именем `project02.txt`.

**ВАЖНО: Для точной проверки сохраняйте порядок и количество строк исходного файла.**

<p style="color:DarkRed"><b>✅ Проект будет засчитан, если вы преодолеете порог 0.1.</b></p>

### Подсказки

1. Обратите внимание на размер датасета.

2. Не забывайте мониторить свои ресурсы: если выполняете long-running jobs, сохраняйте результаты в конце и освобождайте память. Помните, что простаивающие процессы, держащие в памяти гигабайты данных, могут отстреливаться.

3. “Когда в руках молоток, все начинает напоминать гвоздь”. Думайте outside of the box.


<img width="60px" align="left" style="margin-right:20px" src="/extra/images/npl_logo.png"> <br /><b>Желаем вам удачи и успехов!</b>

## Ваше решение здесь